In [1]:
# --- HML Factor Replication and Analysis ---
#
# Objective:
# This notebook replicates the Fama-French Value (HML) factor using the
# Jensen, Kelly, and Pedersen (JKP) global factor dataset. The process involves:
# 1. Loading the pre-processed data.
# 2. Constructing decile portfolios based on the Book-to-Market (be_me) characteristic.
# 3. Calculating the monthly return of a long-short HML portfolio.
# 4. Validating the replicated factor against the Ken French benchmark.
#
# Professional Workflow Notes:
# - All data ingestion is handled by the `src/data_ingestion.py` script.
# - This notebook is for exploratory analysis, portfolio construction, and visualization.
# - The environment is managed via Conda (`qr_env`).

# --- 1. Imports and Setup ---
import polars as pl
from pathlib import Path
import getpass
import wrds

print("--- Notebook Setup Complete ---")

# --- 2. Load the Downloaded Data ---
# Define the path to our data relative to the project root.
# This makes the notebook portable and runnable on any machine.
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    # If running interactively in a notebook, __file__ is not defined.
    # We can find the project root by looking for the .git directory.
    current_dir = Path.cwd()
    while not (current_dir / ".git").exists():
        current_dir = current_dir.parent
    project_root = current_dir

# Path to the raw data file we downloaded
data_path = project_root / "data" / "raw" / "jkp_hml_raw_data_usa.parquet"

print(f"Project Root: {project_root}")
print(f"Loading data from: {data_path}")

# Load the data using Polars
try:
    df = pl.read_parquet(data_path)
    print("Data loaded successfully.")
    print(f"Data Shape: {df.shape}")
except Exception as e:
    print(f"Failed to load data: {e}")
    df = None # Ensure df is defined even on failure
    
# --- 3. Initial Data Inspection (Sanity Check) ---
if df is not None:
    # Display the first few rows to understand the structure
    print("\n--- Data Head (First 5 Rows) ---")
    print(df.head())

    # Display summary statistics for the key columns
    print("\n--- Summary Statistics ---")
    print(df.select(["be_me", "me", "ret_exc_lead1m"]).describe())

    # Check for missing values in our key columns
    print("\n--- Missing Value Counts ---")
    print(df.select(["be_me", "me", "ret_exc_lead1m"]).null_count())


--- Notebook Setup Complete ---
Project Root: /Users/choyounghwan/Documents/GitHub/qr-factor-replication
Loading data from: /Users/choyounghwan/Documents/GitHub/qr-factor-replication/data/raw/jkp_hml_raw_data_usa.parquet
Data loaded successfully.
Data Shape: (4262872, 6)

--- Data Head (First 5 Rows) ---
shape: (5, 6)
┌─────────────────────┬──────────────┬────────┬───────┬────────────┬────────────────┐
│ eom                 ┆ id           ┆ permno ┆ be_me ┆ me         ┆ ret_exc_lead1m │
│ ---                 ┆ ---          ┆ ---    ┆ ---   ┆ ---        ┆ ---            │
│ datetime[ns]        ┆ f64          ┆ f64    ┆ f64   ┆ f64        ┆ f64            │
╞═════════════════════╪══════════════╪════════╪═══════╪════════════╪════════════════╡
│ 1962-02-28 00:00:00 ┆ 1.01127101e8 ┆ null   ┆ null  ┆ null       ┆ null           │
│ 1962-01-31 00:00:00 ┆ 1.00112401e8 ┆ null   ┆ null  ┆ null       ┆ null           │
│ 1962-01-31 00:00:00 ┆ 1.00115301e8 ┆ null   ┆ null  ┆ null       ┆ null     

In [2]:
# --- 4. Data Cleaning and Preparation ---
#
# Professional Rationale:
# We follow the methodology outlined in the JKP documentation and paper.
# 1. Drop nulls: We require a valid characteristic (be_me), forward return (ret_exc_lead1m),
#    and market equity (me) for our analysis.
# 2. Exclude microcaps: The JKP paper (Section 2) and documentation (Section 2)
#    state that portfolio breakpoints are based on "non-micro stocks" (those larger
#    than the NYSE 20th percentile). We will apply this crucial screen.

if df is not None:
    print("\n--- Cleaning Data ---")
    initial_rows = df.shape[0]
    print(f"Initial row count: {initial_rows:,}")

    # Step 4.1: Drop rows with nulls in the columns essential for our analysis
    df_filtered = df.drop_nulls(subset=["be_me", "me", "ret_exc_lead1m"])
    
    rows_after_null_drop = df_filtered.shape[0]
    print(f"Row count after dropping nulls: {rows_after_null_drop:,}")
    
    # Add the size group column from the raw data before dropping more rows
    # The 'size_grp' column is pre-calculated by the JKP authors.
    # We need to pull it from the original 'df' based on the 'id' and 'eom'
    # We will need the original dataframe with the size_grp column. Let's re-run the ingestion to include it.
    # For now, let's assume we have it and proceed.
    # NOTE: This part highlights the iterative process of research. We will update the ingestion script later.
    
    # Step 4.2: Exclude micro-cap stocks as per the JKP methodology
    # The documentation defines 'small' caps as stocks above the 20th percentile.
    # Therefore, 'non-micro' includes 'small', 'large', and 'mega' caps.
    # We need to know the values in the 'size_grp' column. Let's assume for now they are strings.
    # This step is a placeholder until we add 'size_grp' to our data.
    
    # For the purpose of this exercise, let's add a placeholder for the size filter
    # In a real scenario, we would go back and modify the data ingestion script first.
    
    # df_cleaned = df_filtered.filter(
    #     pl.col("size_grp").is_in(["small", "large", "mega"])
    # )
    
    # Since we don't have the 'size_grp' column yet, for now our cleaned df is just the filtered one.
    # This is an important point to note for the next step.
    df_cleaned = df_filtered
    
    cleaned_rows = df_cleaned.shape[0]
    print(f"Final cleaned row count: {cleaned_rows:,}")
    print(f"Total rows dropped: {initial_rows - cleaned_rows:,} ({ (initial_rows - cleaned_rows) / initial_rows:.2%})")

    # Verify that there are no more nulls in these columns
    print("\n--- Missing Value Counts After Cleaning ---")
    print(df_cleaned.select(["be_me", "me", "ret_exc_lead1m"]).null_count())

    print("\n--- Cleaned Data Head ---")
    print(df_cleaned.head().sort("eom"))


--- Cleaning Data ---
Initial row count: 4,262,872
Row count after dropping nulls: 3,341,501
Final cleaned row count: 3,341,501
Total rows dropped: 921,371 (21.61%)

--- Missing Value Counts After Cleaning ---
shape: (1, 3)
┌───────┬─────┬────────────────┐
│ be_me ┆ me  ┆ ret_exc_lead1m │
│ ---   ┆ --- ┆ ---            │
│ u32   ┆ u32 ┆ u32            │
╞═══════╪═════╪════════════════╡
│ 0     ┆ 0   ┆ 0              │
└───────┴─────┴────────────────┘

--- Cleaned Data Head ---
shape: (5, 6)
┌─────────────────────┬──────────────┬────────┬──────────┬─────────────┬────────────────┐
│ eom                 ┆ id           ┆ permno ┆ be_me    ┆ me          ┆ ret_exc_lead1m │
│ ---                 ┆ ---          ┆ ---    ┆ ---      ┆ ---         ┆ ---            │
│ datetime[ns]        ┆ f64          ┆ f64    ┆ f64      ┆ f64         ┆ f64            │
╞═════════════════════╪══════════════╪════════╪══════════╪═════════════╪════════════════╡
│ 1962-01-31 00:00:00 ┆ 1.00123901e8 ┆ null   ┆ 0.016